# Generate the agent
This notebook latter we will pass it into the `base agents.py`

In [ ]:
#Imports
import os
import json

#Typing
from typing import Dict, List, Any, Optional, TypedDict

#LangGraph/LangChain
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI  # or any other LLM provider
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

# Prompts
#Load env files
from dotenv import load_dotenv
load_dotenv()

import io
# At the top of your notebook or script
import nest_asyncio
nest_asyncio.apply()



## Systems Prompts


In [43]:
# Main Agent Prompts (largely unchanged since it's the orchestrator)
MAIN_AGENT_PROMPT = """You are an AI orchestrator for a story processing system. 
Analyze the user's request and the provided text to determine:
1. Is this an Aesop's fable? (yes/no)
2. What action should be taken? (analyze/retell/expand/modernize/create_new)
3. Any specific requirements? (style, moral, characters)

Respond in JSON format with keys: is_aesop, action, requirements"""

# Aesop Tool Prompts - Enhanced Analysis
ANALYZE_FABLE_PROMPT = """You are an expert in Aesop's fables and modern storytelling. Analyze the given fable and provide:
1. The core moral/lesson (what universal truth is it teaching?)
2. The conflict pattern (what fundamental challenge/dilemma does it present?)
3. Character archetypes and their essential traits (what roles do they serve?)
4. The narrative structure (setup, challenge, resolution)
5. What makes this moral relevant today

Format your response as JSON with these keys: moral, conflict_pattern, characters, structure, modern_relevance"""

# Enhanced Brainstorming for Modern Micro-Fables
BRAINSTORM_STORY_PROMPT = """You are a creative storyteller specializing in ultra-short modern fables for social media.
Based on the analysis provided, brainstorm ideas for a 100-110 word modern fable:

1. Fresh animal substitutions: Replace the original animals with unexpected species that maintain the same character traits but feel more surprising (consider unusual animals from diverse ecosystems)

2. Innovative settings: Suggest 2-3 unique settings beyond traditional forests (coral reefs, urban environments, cosmic settings, etc.)

3. Modern context: How could the core conflict be reframed in a contemporary or unexpected context while preserving the moral?

4. Implicit teaching approach: How to convey the moral without explicitly stating it, followed by a memorable 5-10 word takeaway phrase

Format your response as JSON with these keys: animal_substitutions, settings, modern_context, implicit_teaching, takeaway_phrase"""

# Enhanced Story Generation for TikTok-Style Fables
GENERATE_STORY_AESOP_PROMPT = """You are a master of micro-storytelling creating modern Aesop fables for the TikTok generation.
Create a compelling 120-130 word fable that:

1. Uses the suggested animal substitutions and innovative setting
2. Presents a complete narrative arc (setup, conflict, resolution) with extreme efficiency
3. Teaches the moral implicitly through the story
4. Uses vivid, sensory language that creates mental images
5. Ends with the suggested 5-10 word takeaway phrase (not an explicit "the moral is...")

STRICT CONSTRAINTS:
- Exactly 110-100 words for the main story
- Takeaway phrase should be 5-10 words, positioned at the end
- No explicit statement of "the moral is..." or "this teaches us..." in the story
- Every word must serve multiple purposes (character, plot, and theme)

Your task is to distill ancient wisdom into a shareable modern micro-fable."""

# Enhanced Output Formatting
FORMAT_OUTPUT_PROMPT = """Format this modern Aesop fable for maximum impact on social platforms:

1. Present the story as a complete micro-fable (exactly as generated, preserving the 100-110 word count)
2. Set the takeaway phrase on its own line at the end, styled for emphasis
3. Include a brief note about the original fable it's based on
4. Mention one interesting insight about how this modern version preserves the timeless wisdom

Keep the entire output concise and visually scannable - perfect for a quick digital read."""

# pRompt for image generation
IMAGE_PROMPT_GENERATOR_PROMPT = """You are a master prompt engineer for AI image generation models like DALL-E 3, Midjourney and Stable Diffusion.

    Create 8-10 highly detailed, evocative image prompts for key moments in this fable. Each prompt should:
    
    1. Focus on a specific, emotionally resonant moment from the story
    2. Include rich character details (expressions, postures, actions)
    3. Describe environmental elements that enhance the scene
    4. Specify lighting, atmosphere, and mood
    5. Include artistic style direction (storybook illustration, fairytale, etc.)
    6. Mention color palette and composition
    7. Include symbolic elements that reinforce the moral
    8. Be 3-5 sentences long, with incredible detail
    9. Have clear emotional impact
    
    Format each prompt as:
    
    SCENE #: [Title]
    [Detailed, evocative image prompt with all elements above]
    
    Examples of excellent prompts:
    "A close-up of a chubby fox face inside a hollow tree, eyes closed in thought, waiting patiently—symbolic of reflection and growth. Magical lighting filters through knotholes, illuminating the rustic forest background with golden rays. Vibrant fairytale woodland setting with moss and tiny mushrooms framing the scene. Expressive character design with detailed fur textures and a peaceful expression."
    
    "A majestic eagle with sharp, piercing eyes and outstretched talons swoops down from a dark, stormy sky, its wings spread wide, casting a dramatic shadow on the ancient, moss-covered stones beneath, as it grasps a juicy, smoldering piece of meat. The flickering flame illuminates the intense, primal scene, surrounded by eerie, twisted trees with gnarled branches like withered fingers. Rendered in a vibrant, fairytale illustration style with rich, bold lines, intricate textures, and a muted, earthy color palette, evoking foreboding and symbolic greed."
    
    Create prompts that would produce a cohesive visual narrative across all images if generated in sequence."""

## Create metadata class

In [35]:
import time
import json
import functools

from datetime import datetime
import tiktoken  # For token counting

# Global metadata store
metadata = {
    "session_start": time.time(),
    "session_id": datetime.now().strftime("%Y%m%d_%H%M%S"),
    "current_story": None,
    "stories": {},
    "total_tokens": 0,
    "total_cost": 0,
    "total_time": 0
}

def track_node(node_name, tool_name="main"):
    """Decorator to track execution time of nodes"""
    def decorator(func):
        @functools.wraps(func)
        def wrapper(state):
            # Get current story ID or create one
            story_id = metadata.get("current_story")
            if not story_id:
                story_id = f"story_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
                metadata["current_story"] = story_id
                metadata["stories"][story_id] = {
                    "start_time": time.time(),
                    "nodes": {},
                    "tools": {},
                    "llm_calls": [],
                    "total_tokens": 0,
                    "total_cost": 0
                }
            
            # Initialize node data if needed
            story = metadata["stories"][story_id]
            if node_name not in story["nodes"]:
                story["nodes"][node_name] = {
                    "calls": 0,
                    "total_time": 0,
                    "tokens": 0
                }
            
            # Start timing
            start_time = time.time()
            
            # Call the function
            try:
                result = func(state)
                # Record timing data
                end_time = time.time()
                duration = end_time - start_time
                
                # Update metrics
                story["nodes"][node_name]["calls"] += 1
                story["nodes"][node_name]["total_time"] += duration
                
                print(f"Node {node_name} executed in {duration:.2f} seconds")
                
                return result
            except Exception as e:
                # Record error but re-raise
                end_time = time.time()
                duration = end_time - start_time
                
                print(f"Error in {node_name}: {str(e)}")
                story["nodes"][node_name]["calls"] += 1
                story["nodes"][node_name]["total_time"] += duration
                story["nodes"][node_name]["errors"] = story["nodes"][node_name].get("errors", 0) + 1
                
                raise
        return wrapper
    return decorator

def track_llm_call(node_name, tool_name, model, system_prompt, user_prompt, response_text):
    """Track an LLM API call"""
    story_id = metadata.get("current_story")
    if not story_id:
        return
    
    # Simple token estimation (very rough)
    input_tokens = (len(system_prompt) + len(user_prompt)) // 4  # ~4 chars per token
    output_tokens = len(response_text) // 4
    
    # Cost estimation (very rough)
    if model == "gpt-4.1-mini":
        input_cost = (input_tokens / 1000) * 0.00015
        output_cost = (output_tokens / 1000) * 0.00060
    else:
        input_cost = (input_tokens / 1000) * 0.00010
        output_cost = (output_tokens / 1000) * 0.00030
    
    total_cost = input_cost + output_cost
    
    # Record the call
    call_data = {
        "timestamp": time.time(),
        "node": node_name,
        "tool": tool_name,
        "model": model,
        "input_tokens": input_tokens,
        "output_tokens": output_tokens,
        "total_tokens": input_tokens + output_tokens,
        "total_cost": total_cost
    }
    
    # Update story
    story = metadata["stories"][story_id]
    story["llm_calls"].append(call_data)
    story["total_tokens"] += input_tokens + output_tokens
    story["total_cost"] += total_cost
    
    # Update node
    if node_name in story["nodes"]:
        story["nodes"][node_name]["tokens"] += input_tokens + output_tokens
    
    # Update global
    metadata["total_tokens"] += input_tokens + output_tokens
    metadata["total_cost"] += total_cost
    
    print(f"LLM call in {node_name}: {input_tokens + output_tokens} tokens, ${total_cost:.4f}")

def finish_story(output_text):
    """Finish tracking the current story"""
    story_id = metadata.get("current_story")
    if not story_id:
        return
    
    story = metadata["stories"][story_id]
    story["end_time"] = time.time()
    story["duration"] = story["end_time"] - story["start_time"]
    story["output_length"] = len(output_text)
    
    # Calculate summary stats
    total_time = sum(node["total_time"] for node in story["nodes"].values())
    total_calls = sum(node["calls"] for node in story["nodes"].values())
    
    print("\n=== STORY METRICS ===")
    print(f"Total execution time: {story['duration']:.2f} seconds")
    print(f"Total tokens: {story['total_tokens']} tokens")
    print(f"Estimated cost: ${story['total_cost']:.4f}")
    print(f"Number of nodes executed: {len(story['nodes'])}")
    print(f"Number of LLM calls: {len(story['llm_calls'])}")
    
    # Reset current story
    metadata["current_story"] = None
    metadata["total_time"] += story["duration"]
    
    # Export metadata
    with open(f"story_metrics_{story_id}.json", "w") as f:
        json.dump(story, f, indent=2, default=str)
    
    print(f"Metrics saved to story_metrics_{story_id}.json")
    
    return story

## Step 1:
Import OPENAI as a global variable and define it.

In [36]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Call the api keys from the .env file
llm_openai_41_mini = ChatOpenAI(model="gpt-4.1-mini")

## Step 2:
Define the the states of the main graph and the subgraphs

In [37]:
class MainState(TypedDict):
    messages: List[Dict[str, Any]]  # User messages
    current_fable: str              # Input fable text
    tool_to_call: str               # Which tool subgraph to use
    processing_request: Dict        # Request info for the tool
    tool_output: Dict               # Output from selected tool
    final_story: str                # Final output after post-processing
    image_prompts: List[Dict]       # List of image prompts for visualization

# Cell 3: Define Aesop State (for subgraph)
class AesopState(TypedDict):
    original_fable: str             # Original input text
    analysis: Dict                  # Analysis of the fable (moral, characters, etc)
    brainstorm: Dict                # Ideas for the story creation/modification
    generated_story: str            # The final story created by this tool

## Step 3:
Create the story router

In [ ]:

# Cell 4: Main Agent for the main graph
@track_node("main_agent", "main")
def main_agent(state: MainState) -> Dict[str, Any]:
    """
    Main orchestrator that analyzes input and decides which tool to use
    """
    print("\n=== Main Agent ===")
    current_fable = state.get("current_fable", "")
    messages = state.get("messages", [])
    
    user_message = messages[-1].get("content", "") if messages else ""
    print(f"Processing request: {user_message}")
    print(f"Current fable length: {len(current_fable)} characters")
    
    is_aesop = True
    
    if is_aesop:
        processing_request = {
            "user_intent": user_message,
            "fable_text": current_fable
        }
        
        result = {
            "processing_request": processing_request,
            "tool_to_call": "aesop_tool"
        }
        print(f"Selecting tool: aesop_tool")
        return result
    else:
        # Future expansion for other tools
        return {
            "processing_request": {},
            "tool_to_call": "generic_tool" 
        }

# Cell 5: Tool Router
def tool_router(state: MainState) -> Dict[str, Any]:
    """
    Routes to the appropriate tool subgraph
    """
    print("\n=== Tool Router ===")
    tool_name = state.get("tool_to_call", "")
    processing_request = state.get("processing_request", {})
    
    print(f"Routing to tool: {tool_name}")
    
    if tool_name == "aesop_tool":
        # Call the Aesop subgraph
        aesop_result = aesop_subgraph(processing_request)
        print(f"Received result from Aesop tool")
        return {"tool_output": aesop_result}
    else:
        # Future: add more tool subgraphs
        return {"tool_output": {"error": "Tool not found"}}

@track_node("generate_output", "main")
def generate_output(state: MainState) -> Dict[str, Any]:
    """
    Formats the final output based on the tool results
    Creates a two-part story with strict word count limits
    """
    print("\n=== Generate Output ===")
    tool_output = state.get("tool_output", {})
    tool_name = state.get("tool_to_call", "")
    
    if "error" in tool_output:
        final_story = f"Error: {tool_output['error']}"
    else:
        if tool_name == "aesop_tool":
            # Get story components from the tool output
            generated_story = tool_output.get("generated_story", "")
            analysis = tool_output.get("analysis", {})
            brainstorm = tool_output.get("brainstorm", {})
            
            # Track the original story length for metadata
            original_word_count = len(generated_story.split())
            print(f"Original story word count: {original_word_count}")
            
            # Create a prompt to split and enhance the story with strict word counts
            system_prompt = """You are a master storyteller specializing in micro-fables for social media.
            Take this fable and transform it into a two-part story with STRICT word count limits:
            
            PART 1 (First Post):
            - EXACTLY 80-90 WORDS MAXIMUM
            - Contains the setup, characters, and builds tension
            - Ends at a compelling moment that makes readers curious for the conclusion
            - Should be clearly labeled as "PART 1"
            
            PART 2 (Second Post):
            - EXACTLY 80-90 WORDS MAXIMUM (including the recap)
            - Begins with a brief 1-sentence recap of Part 1, and explicitly add at the begginig In Part 1
            - Contains the resolution and moral/takeaway
            - Ends with a memorable phrase that captures the moral
            - Should be clearly labeled as "PART 2"
            
            The word counts are STRICT REQUIREMENTS - do not exceed 80 words for each part.
            Count your words carefully before finalizing each part.
            """
            
            user_prompt = f"""Story: {generated_story}
            
            Analysis: {json.dumps(analysis, indent=2)}
            
            Split this into a two-part story as described, with exactly 80 words maximum for each part.
            Ensure Part 2 begins with a brief recap so viewers can understand the conclusion even if they missed Part 1.
            """
            
            # Call LLM to reformat the story
            response = llm_openai_41_mini.invoke([
                SystemMessage(content=system_prompt),
                HumanMessage(content=user_prompt)
            ])
            
            # Track LLM call
            track_llm_call(
                node_name="generate_output",
                tool_name="main",
                model="gpt-4.1-mini",
                system_prompt=system_prompt,
                user_prompt=user_prompt,
                response_text=response.content
            )
            
            # The final reformatted story
            final_story = response.content
            
            # Track final word count for metadata
            final_word_count = len(final_story.split())
            print(f"Final story word count: {final_word_count}")
            
            # Verify the word counts of each part (for validation and debugging)
            parts = final_story.split("PART 2")
            if len(parts) > 1:
                part1 = parts[0].replace("PART 1", "").strip()
                part2 = "PART 2" + parts[1].strip()
                
                part1_words = len(part1.split())
                part2_words = len(part2.split())
                
                print(f"Part 1 word count: {part1_words}")
                print(f"Part 2 word count: {part2_words}")
                
                # Add to metadata
                if "current_story" in metadata and metadata["current_story"] in metadata["stories"]:
                    story_id = metadata["current_story"]
                    if "transformations" not in metadata["stories"][story_id]:
                        metadata["stories"][story_id]["transformations"] = {}
                    
                    metadata["stories"][story_id]["transformations"]["story_split"] = {
                        "original_word_count": original_word_count,
                        "final_word_count": final_word_count,
                        "part1_word_count": part1_words,
                        "part2_word_count": part2_words,
                        "transformation_type": "two-part-fixed-length",
                        "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    }
                    
                    print("Added transformation metadata")
        else:
            # Future: handle other tool outputs with different formatting strategies
            final_story = str(tool_output)
    
    print(f"Final story generated (length: {len(final_story)} characters)")
    return {"final_story": final_story}


@track_node("image_prompt_generator", "main")
def image_prompt_generator(state: MainState) -> Dict[str, Any]:
    """
    Generates image prompts for key scenes in the two-part story
    """
    print("\n=== Image Prompt Generator ===")
    final_story = state.get("final_story", "")
    tool_output = state.get("tool_output", {})
    
    # Extract analysis for context
    analysis = tool_output.get("analysis", {})
    
    # Split the story into parts
    parts = final_story.split("PART 2")
    if len(parts) < 2:
        parts = [final_story, ""]  # Fallback if not properly split
    
    part1 = parts[0].replace("PART 1", "").strip()
    part2 = "PART 2" + parts[1].strip() if len(parts) > 1 else ""
    
    # Create prompt for generating image descriptions
    system_prompt = IMAGE_PROMPT_GENERATOR_PROMPT
    
    user_prompt = f"""Two-part fable to visualize:

    PART 1:
    {part1}

    PART 2:
    {part2}

    Character information from analysis:
    {json.dumps(analysis.get('characters', {}), indent=2)}

    Generate 8-10 image prompts for key visual moments throughout this story."""
    
    # Call LLM to generate image prompts
    response = llm_openai_41_mini.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_prompt)
    ])
    
    # Track LLM call
    track_llm_call(
        node_name="image_prompt_generator",
        tool_name="main",
        model="gpt-4.1-mini",
        system_prompt=system_prompt,
        user_prompt=user_prompt,
        response_text=response.content
    )
    
    # Process the response into structured prompts
    image_prompts_text = response.content
    
    # Parse the scene prompts
    import re
    scene_pattern = r'SCENE (\d+): ([^\n]+)\n(.*?)(?=SCENE \d+:|$)'
    matches = re.findall(scene_pattern, image_prompts_text, re.DOTALL)
    
    image_prompts = []
    for scene_num, title, description in matches:
        image_prompts.append({
            "scene_number": int(scene_num),
            "title": title.strip(),
            "description": description.strip(),
            "story_part": 1 if int(scene_num) <= len(matches)//2 else 2,  # Rough division between parts
        })
    
    # Add metadata
    if "current_story" in metadata and metadata["current_story"] in metadata["stories"]:
        story_id = metadata["current_story"]
        metadata["stories"][story_id]["image_prompts"] = {
            "count": len(image_prompts),
            "prompts": image_prompts,
            "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
    
    print(f"Generated {len(image_prompts)} image prompts")
    
    # Format a preview of prompts
    preview = "\n\n".join([f"SCENE {p['scene_number']}: {p['title']}" for p in image_prompts])
    print(f"Image prompt scenes:\n{preview}")
    
    # Return augmented state with image prompts
    return {"image_prompts": image_prompts}

## Step 4:
Create the tools-subgraphs, currently we have:
* Aesop tool-subgraph

In [39]:
# Node 1: Analyze Fable with tracking
@track_node("analyze_fable", "aesop_tool")
def analyze_fable(state: AesopState) -> Dict[str, Any]:
    """
    Analyzes the fable structure, characters, and moral
    """
    print("\n== Aesop Subgraph: Analyze Fable ==")
    original_fable = state.get("original_fable", "")
    print(f"Analyzing fable (length: {len(original_fable)} characters)")
    
    # Use LLM to analyze the fable with prompt from prompts.py
    system_prompt = ANALYZE_FABLE_PROMPT
    
    user_prompt = f"Analyze this fable:\n\n{original_fable}"
    
    response = llm_openai_41_mini.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_prompt)
    ])
    
    # Track the LLM call
    track_llm_call(
        node_name="analyze_fable",
        tool_name="aesop_tool",
        model="gpt-4.1-mini",
        system_prompt=system_prompt,
        user_prompt=user_prompt,
        response_text=response.content
    )
    
    try:
        analysis = json.loads(response.content)
    except:
        # Fallback if JSON parsing fails
        print("JSON parsing failed, using content as analysis")
        analysis = {
            "moral": response.content,
            "characters": [],
            "structure": {},
            "symbols": []
        }
    
    print(f"Analysis complete: identified moral '{analysis.get('moral', '')[:50]}...'")
    return {"analysis": analysis}

@track_node("brainstorm_story", "aesop_tool")
# Node 2: Brainstorm Story
def brainstorm_story(state: AesopState) -> Dict[str, Any]:
    """
    Brainstorms ideas for story creation based on analysis
    """
    print("\n== Aesop Subgraph: Brainstorm Story ==")
    original_fable = state.get("original_fable", "")
    analysis = state.get("analysis", {})
    
    print(f"Brainstorming based on analysis of moral: {analysis.get('moral', '')[:50]}...")
    
    response = llm_openai_41_mini.invoke([
        SystemMessage(content=BRAINSTORM_STORY_PROMPT),
        HumanMessage(content=f"Original fable: {original_fable}\n\nAnalysis: {json.dumps(analysis, indent=2)}")
    ])
    
    try:
        brainstorm = json.loads(response.content)
    except:
        print("JSON parsing failed, using content as brainstorm")
        brainstorm = {
            "moral_approaches": response.content,
            "variations": [],
            "character_ideas": [],
            "imagery": []
        }
    
    print(f"Brainstorming complete: generated {len(brainstorm.keys())} idea categories")
    return {"brainstorm": brainstorm}


@track_node("generate_story_aessop", "aesop_tool")
# Node 3: Generate Story
def generate_story_aessop(state: AesopState) -> Dict[str, Any]:
    """
    Generates the final story based on analysis and brainstorming
    """
    print("\n== Aesop Subgraph: Generate Story ==")
    original_fable = state.get("original_fable", "")
    analysis = state.get("analysis", {})
    brainstorm = state.get("brainstorm", {})
    
    print(f"Generating story based on analysis and brainstorming")
    
    response = llm_openai_41_mini.invoke([
        SystemMessage(content=GENERATE_STORY_AESOP_PROMPT),
        HumanMessage(content=f"""
        Original fable: {original_fable}
        
        Analysis: {json.dumps(analysis, indent=2)}
        
        Brainstorming: {json.dumps(brainstorm, indent=2)}
        
        Create a refined version of this fable.
        """)
    ])
    
    generated_story = response.content
    print(f"Story generated (length: {len(generated_story)} characters)")
    
    return {"generated_story": generated_story}

# Cell 8: BUILD THE AESOP SUBGRAPH
def build_aesop_subgraph():
    """
    Builds the Aesop tool subgraph
    """
    builder = StateGraph(AesopState)
    
    # Add nodes
    builder.add_node("analyze_fable", analyze_fable)
    builder.add_node("brainstorm_story", brainstorm_story)
    builder.add_node("generate_story_aessop", generate_story_aessop)
    
    # Add edges
    builder.add_edge(START, "analyze_fable")
    builder.add_edge("analyze_fable", "brainstorm_story")
    builder.add_edge("brainstorm_story", "generate_story_aessop")
    builder.add_edge("generate_story_aessop", END)
    
    graph = builder.compile()

    # # Generate the graph
    # mermaid_graph = graph.get_graph().draw_mermaid_png(
    #     draw_method=MermaidDrawMethod.PYPPETEER,
    #     output_file_path="./graphs_images/aesop_subgraph.png"  # Specify where to save
    # )

    # Compile
    return builder.compile()

# Cell 9: AESOP SUBGRAPH WRAPPER
def aesop_subgraph(processing_request: Dict) -> Dict:
    """
    Wrapper function that runs the Aesop subgraph
    """
    print("\n=== Running Aesop Subgraph ===")
    
    # Create initial state for Aesop subgraph
    initial_state = {
        "original_fable": processing_request.get("fable_text", ""),
        "analysis": {},
        "brainstorm": {},
        "generated_story": ""
    }
    
    # Build and run the subgraph
    aesop_graph = build_aesop_subgraph()
    result = aesop_graph.invoke(initial_state)
    
    # Return the enriched state
    return {
        "analysis": result["analysis"],
        "brainstorm": result["brainstorm"],
        "generated_story": result["generated_story"]
    }

## Step 5:
Build the main graph

In [ ]:

# Cell 10: BUILD THE MAIN GRAPH
def decide_next_step(state: MainState) -> str:
    """
    Decide whether to generate image prompts based on tool type
    """
    tool_name = state.get("tool_to_call", "")
    
    if tool_name == "aesop_tool":
        return "image_prompt_generator"
    else:
        return END

def build_main_graph():
    """
    Builds the main orchestrator graph with image prompt generation
    """
    builder = StateGraph(MainState)
    
    # Add nodes
    builder.add_node("main_agent", main_agent)
    builder.add_node("tool_router", tool_router)
    builder.add_node("generate_output", generate_output)
    builder.add_node("image_prompt_generator", image_prompt_generator)
    
    # Add edges
    builder.add_edge(START, "main_agent")
    builder.add_edge("main_agent", "tool_router")
    builder.add_edge("tool_router", "generate_output")
    
    # Conditional edge after generate_output
    builder.add_conditional_edges(
        "generate_output",
        decide_next_step,
        {
            "image_prompt_generator": "image_prompt_generator",
            END: END
        }
    )
    
    # Final edge
    builder.add_edge("image_prompt_generator", END)
    
    # Compile
    graph = builder.compile()

    # # Generate the graph
    # mermaid_graph = graph.get_graph().draw_mermaid_png(
    #     draw_method=MermaidDrawMethod.PYPPETEER,
    #     output_file_path="./graphs_images/main_graph.png"  # Specify where to save
    # )

    return graph



In [51]:
def test_system():
    # Build the main graph
    main_graph = build_main_graph()
    print("Graph built successfully!")
    
    # Test with a sample Aesop fable
    test_fable = """
    La zorra y el mono coronado
    rey
    Uso Educacional
    En una junta de animales, bailó tan bonito el mono,
    que ganándose la simpatía de los espectadores,
    fue elegido rey.
    Celosa la zorra por no haber sido ella la elegida, vio un trozo de
    comida en un cepo y llevó allí al mono, diciéndole que había
    encontrado un tesoro digno de reyes, pero que en lugar de tomarlo
    para llevárselo a él, lo había guardado para que fuera él
    personalmente quien lo cogiera, ya que era una prerrogativa real.
    El mono se acercó sin más reflexión,
    y quedó prensado en el cepo.
    Entonces la zorra, a quien el mono acusaba de
    tenderle aquella trampa, repuso:
    -- ¡Eres muy tonto, mono, y todavía pretendes reinar
    entre todos los animales!
    No te lances a una empresa, si ant es no
    has reflexionado sobre sus posibles éxitos
    o peligros.    
    """
    
    initial_state = {
        "messages": [{"role": "user", "content": "Analyze and enhance this fable"}],
        "current_fable": test_fable,
        "tool_to_call": "",
        "processing_request": {},
        "tool_output": {},
        "final_story": "",
        "image_prompts": []
    }
    
    # Run the graph
    result = main_graph.invoke(initial_state)
    
    # Print the results
    print("\n=== FINAL STORY ===")
    print(result["final_story"])
    
    print("\n=== IMAGE PROMPTS ===")
    for prompt in result.get("image_prompts", []):
        print(f"\nSCENE {prompt['scene_number']}: {prompt['title']}")
        print(prompt['description'])
    
    # Finish tracking this story
    story_stats = finish_story(result["final_story"])
    
    return result

## Step 6:
Test the system

In [52]:
# Cell to actually run the test
test_result = test_system()

Graph built successfully!

=== Main Agent ===
Processing request: Analyze and enhance this fable
Current fable length: 891 characters
Selecting tool: aesop_tool
Node main_agent executed in 0.00 seconds

=== Tool Router ===
Routing to tool: aesop_tool

=== Running Aesop Subgraph ===

== Aesop Subgraph: Analyze Fable ==
Analyzing fable (length: 891 characters)
LLM call in analyze_fable: 795 tokens, $0.0003
JSON parsing failed, using content as analysis
Analysis complete: identified moral '```json
{
  "moral": "Do not rush into actions or ...'
Node analyze_fable executed in 6.97 seconds

== Aesop Subgraph: Brainstorm Story ==
Brainstorming based on analysis of moral: ```json
{
  "moral": "Do not rush into actions or ...
Brainstorming complete: generated 5 idea categories
Node brainstorm_story executed in 5.48 seconds

== Aesop Subgraph: Generate Story ==
Generating story based on analysis and brainstorming
Story generated (length: 606 characters)
Node generate_story_aessop executed in 3.5

In [1]:
from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO
import os

In [2]:
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
client = genai.Client(api_key=GEMINI_API_KEY)

In [3]:
import json
import os
from pathlib import Path
from google import genai
from google.genai import types
import time

def load_story_metadata(json_file_path):
    """Load the story metadata from JSON file"""
    with open(json_file_path, 'r') as f:
        data = json.load(f)
    return data

def generate_images_for_story(json_file_path, output_dir="generated_images", overwrite=False):
    """
    Generate images from the JSON file for any story structure
    
    Args:
        json_file_path: Path to the JSON file
        output_dir: Directory to save images
        overwrite: If True, overwrites existing images. If False, skips existing files.
    """
    print(f"Loading story data from: {json_file_path}")
    
    # # Setup - you'll need to set your API key
    # GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY")
    # if not GEMINI_API_KEY:
    #     raise ValueError("Please set GOOGLE_API_KEY in your environment variables")
    
    client = genai.Client(api_key=GEMINI_API_KEY)
    
    # Load the story data
    story_data = load_story_metadata(json_file_path)
    
    # MADE GENERAL: Handle different JSON structures
    prompts_data = None
    story_id = None
    
    # Check if this is the new structure with 'stories' key
    if "stories" in story_data:
        # Get the first (and likely only) story
        stories = story_data["stories"]
        if not stories:
            print("No stories found in the JSON file")
            return
        
        # Get the first story ID
        story_id = list(stories.keys())[0]
        story_info = stories[story_id]
        
        if "image_prompts" in story_info:
            prompts_data = story_info["image_prompts"]
            print(f"Found story: {story_id}")
    
    # Check if this is the old flat structure
    elif "image_prompts" in story_data:
        prompts_data = story_data["image_prompts"]
        story_id = Path(json_file_path).stem  # Use filename as story ID
        print(f"Using flat structure for story: {story_id}")
    
    else:
        print("No image prompts found in the JSON file")
        return
    
    prompts = prompts_data.get("prompts", [])
    
    if not prompts:
        print("No prompts found in image_prompts")
        return
        
    print(f"Found {len(prompts)} image prompts to generate")
    
    # Create output directory
    story_dir = Path(output_dir) / story_id
    story_dir.mkdir(parents=True, exist_ok=True)
    
    # Track statistics
    generated_count = 0
    skipped_count = 0
    error_count = 0
    
    # Generate images for each prompt
    for i, prompt_data in enumerate(prompts):
        scene_number = prompt_data.get("scene_number", i+1)
        title = prompt_data.get("title", f"Scene {scene_number}")
        prompt_text = prompt_data.get("description", "")
        story_part = prompt_data.get("story_part", 1)
        
        print(f"\nGenerating Scene {scene_number}: {title}")
        print(f"Part {story_part} | Prompt preview: {prompt_text[:100]}...")
        
        try:
            # Generate images using Imagen 3
            response = client.models.generate_images(
                model='imagen-3.0-generate-002',
                prompt=prompt_text,
                config=types.GenerateImagesConfig(
                    number_of_images=1,
                    aspectRatio="9:16"
                )
            )
            
            # Save the generated images
            for img_idx, generated_image in enumerate(response.generated_images):
                # Create filename
                safe_title = "".join(c for c in title if c.isalnum() or c in (' ', '-', '_')).rstrip()
                filename = f"scene_{scene_number:02d}_{safe_title}_v{img_idx+1}.png"
                filepath = story_dir / filename
                
                # CHECK IF FILE EXISTS AND HANDLE OVERWRITE
                if filepath.exists() and not overwrite:
                    print(f"  ⏭ Skipped (already exists): {filepath}")
                    skipped_count += 1
                    continue
                
                # Save the image
                generated_image.image.save(str(filepath))
                
                if filepath.exists() and not overwrite:
                    print(f"  ✓ Generated: {filepath}")
                else:
                    print(f"  ✓ Overwritten: {filepath}")
                generated_count += 1
            
            # Add a small delay to avoid rate limiting
            time.sleep(2)
            
        except Exception as e:
            print(f"✗ Error generating image for Scene {scene_number}: {str(e)}")
            error_count += 1
            continue
    
    # Print summary
    print(f"\n🎉 Image generation complete!")
    print(f"📁 Images saved to: {story_dir}")
    print(f"📊 Summary:")
    print(f"   - Generated: {generated_count} images")
    print(f"   - Skipped: {skipped_count} images")
    print(f"   - Errors: {error_count} scenes")

def list_stories_in_json(json_file_path):
    """
    List all stories available in the JSON file
    """
    story_data = load_story_metadata(json_file_path)
    
    if "stories" in story_data:
        stories = story_data["stories"]
        print(f"Found {len(stories)} story(ies):")
        for story_id, story_info in stories.items():
            prompt_count = story_info.get("image_prompts", {}).get("count", 0)
            duration = story_info.get("duration", 0)
            print(f"  - {story_id}: {prompt_count} prompts, {duration:.1f}s duration")
    else:
        print("Single story format detected")
        prompt_count = story_data.get("image_prompts", {}).get("count", 0)
        print(f"  - {prompt_count} prompts available")

# Example usage
if __name__ == "__main__":
    # List what stories are available
    list_stories_in_json("story_metrics_story_20250602_220509.json")
    
    # Generate images (won't overwrite by default)
    generate_images_for_story("story_metrics_story_20250602_220509.json", overwrite=True)
        # generate_images_for_story("story_metrics_story_20250602_220509.json", overwrite=True)

Found 1 story(ies):
  - story_20250602_220509: 8 prompts, 52.7s duration
Loading story data from: story_metrics_story_20250602_220509.json
Found story: story_20250602_220509
Found 8 image prompts to generate

Generating Scene 1: The Coral Reef Council Gathering
Part 1 | Prompt preview: A vibrant underwater coral reef scene bustling with diverse sea creatures assembled around an ancien...
  ✓ Overwritten: generated_images/story_20250602_220509/scene_01_The Coral Reef Council Gathering_v1.png
  ✓ Overwritten: generated_images/story_20250602_220509/scene_01_The Coral Reef Council Gathering_v2.png

Generating Scene 2: Zorra’s Whispered Deception
Part 1 | Prompt preview: A close, intense scene focused on Zorra the octopus in a mysterious reef grotto illuminated only by ...
  ✓ Overwritten: generated_images/story_20250602_220509/scene_02_Zorras Whispered Deception_v1.png
  ✓ Overwritten: generated_images/story_20250602_220509/scene_02_Zorras Whispered Deception_v2.png

Generating Scene 3: Mo